In [22]:
import os
import csv
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from contextlib import contextmanager

In [16]:
def get_files(method:str):
    path = f'../workspace/accuracy/{method}/'
    file_list = os.listdir(path)
    file_list_csv = [file for file in file_list if file.endswith('.csv')]
    seeds = []
    pattern = f'{method}_sparse_output_([0-9]+).csv'
    for file in file_list_csv:
        result = re.search(pattern, file)
        seeds.append(result.group(1))
    return path, file_list_csv, seeds

In [20]:
def data_sorting(path, file_str):
    data = pd.read_csv(path+file_str)
    data_sorted = data.sort_values(by="sparsity").drop_duplicates(subset=['sparsity'], keep = 'first')
    return data_sorted

In [28]:
def save_graphs(method:str):
    path, file_list, seeds = get_files(method)
    for i in range(len(file_list)):
        data = data_sorting(path, file_list[i])
        sparsity = np.array(data["sparsity"])
        accuracy = np.array(data["accuracy"])
        plt.plot(sparsity,accuracy)
        plt.xlabel("sparsity")
        plt.ylabel("accuracy")
        plt.title(f"{method}_{seeds[i]}")
        plt.savefig(f"{path}{method}_{seeds[i]}.png")
        plt.clf()

In [30]:
save_graphs('block')

In [32]:
save_graphs('random')

In [35]:
save_graphs('origin')